<a href="https://colab.research.google.com/github/4dsolutions/clarusway_data_analysis/blob/main/DAwPy_S10_(Working%20with%20Text%20and%20Time%20Data)/DAwPy_S10_Joining_Tables.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a><br/>
[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.org/github/4dsolutions/clarusway_data_analysis/blob/main/DAwPy_S10_%28Working%20with%20Text%20and%20Time%20Data%29/DAwPy_S10_Joining_Tables.ipynb)

________


<p style="text-align: center;"><img src="https://docs.google.com/uc?id=1lY0Uj5R04yMY3-ZppPWxqCr5pvBLYPnV" class="img-fluid" 
alt="CLRSWY"></p>

## <p style="background-color:#FDFEFE; font-family:newtimeroman; color:#9d4f8c; font-size:120%; text-align:center; border-radius:10px 10px;">Way to Reinvent Yourself</p>

In this Notebook, we: 

* develop a small set of tables in pandas
* write some Python code for adding data to at least one of them
* combine DataFrames using pandas `merge` and `join`
* store our tables to an SQLite database.

Let's create a small database consisting of three related tables:

* a roster patients seen by a practice
* patient visits with physicians
* a roster of physicians in the practice

In [1]:
import pandas as pd
import numpy as np

Feeding in a list of tuples, while providing column names in the form of a named Series, results in the tuples going in row-wise i.e. row by row

In [2]:
patients = [
    ("13298","Debbie", "Rose",
     "32 SE Beacon St.", 
     "Portland", "OR", "97214", 
     "503-311-9928"),
    ("12446","Jerry", "Turing",
     "491 NW Shanny St.", 
     "Portland", "OR", "97111", 
     "503-311-7865"),
    ("77650","Bruce", "Flemming",
     "32 SE Beacon St.", 
     "Portland", "OR", "97214", 
     "503-311-9928"),
    ("89765","Susan", "Constanza",
     "8976 NW Circle Court, Apt 2E", 
     "Gresham", "OR", "97211", 
     "503-321-8640"),
    ("56768","Raul", "Sosa",
     "786 NW Couch St.", 
     "Portland", "OR", "97212", 
     "503-311-1018")
]

patients_df = pd.DataFrame(
    data=patients,
    columns = pd.Series(["MR", "FIRSTNM","LASTNM", 
               "STREET", "CITY", "STATE", "ZIPCODE", 
               "PHONE"], name="IDENT")
)

patients_df.set_index("MR", inplace=True)

In [3]:
patients_df

IDENT,FIRSTNM,LASTNM,STREET,CITY,STATE,ZIPCODE,PHONE
MR,,,,,,,
13298,Debbie,Rose,32 SE Beacon St.,Portland,OR,97214,503-311-9928
12446,Jerry,Turing,491 NW Shanny St.,Portland,OR,97111,503-311-7865
77650,Bruce,Flemming,32 SE Beacon St.,Portland,OR,97214,503-311-9928
89765,Susan,Constanza,"8976 NW Circle Court, Apt 2E",Gresham,OR,97211,503-321-8640
56768,Raul,Sosa,786 NW Couch St.,Portland,OR,97212,503-311-1018


In [4]:
patients_df.index

Index(['13298', '12446', '77650', '89765', '56768'], dtype='object', name='MR')

Let's automate a process for adding new patient records.  Rather than make up a medical record number, we'll let Python randomly generate one for us, and making sure it's not already in use...

In [5]:
def get_mr(table=patients_df):
    not_ok = True
    while not_ok:
        mr = str(np.random.randint(10000,100000))
        if mr not in table.index:
            not_ok = False
    return mr

In [6]:
get_mr()

'47641'

In [7]:
template = \
"""
{first} {last}
{street}, 
{city}, {state} {zipcode}
{phone}
"""

def add_patient(table=patients_df):
    not_ok = True
    while not_ok:
        
        #prompt for inputs
        first = input("First? >")
        last = input("Last? >")
        street = input("Street? >")
        city = input("City? >")
        state = input("State? >")
        zipcode = input("Zip code? >")
        phone = input("Phone? >")
        
        # substitute local vars into the template
        print(template.format(**locals()))
        ans = input("OK? (Y/N or Quit): >")
        
        if ans.upper() == "N":    # try again
            continue
        elif ans.upper() == "Y":  # add new info
            not_ok = False
            continue    
        else:                     # escape from loop
            break
        
    else: # not_ok == False
        print("Adding new patient record")
        new_mr = get_mr()
        # create a dict using the local vars we've filled in
        new_rec = pd.Series({"MR": new_mr,
                             "FIRSTNM": first,
                             "LASTNM": last,
                             "STREET": street,
                             "CITY": city,
                             "STATE": state,
                             "ZIPCODE": zipcode,
                             "PHONE": phone})
        # turn the input Series into a DataFrame with the same cols and index
        bottom_row = pd.DataFrame(new_rec).T.set_index("MR")
        return pd.concat([table, bottom_row]) # append new row
    # break (above) takes us here         
    print("No action taken")
    return table # return table as received

In [8]:
# newtable  = add_patient()

In [9]:
# newtable

In [10]:
physicians_df = pd.DataFrame(
    {"DR_ID": ["1001", "1002", "1003"],
     "DR_NAME": ["Sheela Morley, M.D.",
                 "Malcolm Head, D.O.",
                 "Patricia Lord, M.D."]}).set_index("DR_ID")

In [11]:
physicians_df

,DR_NAME
DR_ID,
1001,"Sheela Morley, M.D."
1002,"Malcolm Head, D.O."
1003,"Patricia Lord, M.D."


In [12]:
pd.Timedelta(1.5, unit='h')

Timedelta('0 days 01:30:00')

In [13]:
visits = [('77650', '1001', '2023-5-17T13:50', "1.25"),
          ('77650', '1001', '2023-5-31T14:00', "0.75"),
          ('12446', '1003', '2023-5-31T10:15', "0.10"),
          ('89765', '1002', '2023-6-04T10:00', "0.50"),
          ('12446', '1003', '2023-6-04T10:15', "0.10"),]

visits_df = pd.DataFrame(
    data=visits,
    columns = pd.Series(["MR", "DR_ID","CHECK_IN", "DURATION"], name="VISIT")
)

visits_df["CHECK_IN"] = visits_df["CHECK_IN"].astype(np.datetime64)
visits_df["DURATION"] = visits_df["DURATION"].astype(float)
visits_df.style.format({"DURATION":lambda x: pd.Timedelta(x, unit='h').isoformat()[4:9]})

VISIT,MR,DR_ID,CHECK_IN,DURATION
0,77650,1001,2023-05-17 13:50:00,1H15M
1,77650,1001,2023-05-31 14:00:00,0H45M
2,12446,1003,2023-05-31 10:15:00,0H6M0
3,89765,1002,2023-06-04 10:00:00,0H30M
4,12446,1003,2023-06-04 10:15:00,0H6M0


In [14]:
visits_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   MR        5 non-null      object        
 1   DR_ID     5 non-null      object        
 2   CHECK_IN  5 non-null      datetime64[ns]
 3   DURATION  5 non-null      float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 292.0+ bytes


In [15]:
pd.merge(left=visits_df[visits_df.DR_ID == '1003'], 
         right=patients_df[["FIRSTNM", "LASTNM", "PHONE"]], 
         on="MR")

,MR,DR_ID,CHECK_IN,DURATION,FIRSTNM,LASTNM,PHONE
0,12446,1003,2023-05-31 10:15:00,0.1,Jerry,Turing,503-311-7865
1,12446,1003,2023-06-04 10:15:00,0.1,Jerry,Turing,503-311-7865


Might we use `join` to accomplish the same thing?

In [16]:
visits_df[visits_df.DR_ID == '1003'].join( 
         patients_df[["FIRSTNM", "LASTNM", "PHONE"]], 
         on="MR")

,MR,DR_ID,CHECK_IN,DURATION,FIRSTNM,LASTNM,PHONE
2,12446,1003,2023-05-31 10:15:00,0.1,Jerry,Turing,503-311-7865
4,12446,1003,2023-06-04 10:15:00,0.1,Jerry,Turing,503-311-7865


In [17]:
visit_by_doc_df = pd.merge(
                    left=physicians_df, 
                    right=visits_df,
                    how="right",
                    on="DR_ID", 
                    sort=True)
visit_by_doc_df

,DR_ID,DR_NAME,MR,CHECK_IN,DURATION
0,1001,"Sheela Morley, M.D.",77650,2023-05-17 13:50:00,1.25
1,1001,"Sheela Morley, M.D.",77650,2023-05-31 14:00:00,0.75
2,1002,"Malcolm Head, D.O.",89765,2023-06-04 10:00:00,0.50
3,1003,"Patricia Lord, M.D.",12446,2023-05-31 10:15:00,0.10
4,1003,"Patricia Lord, M.D.",12446,2023-06-04 10:15:00,0.10


In [18]:
visit_by_doc_df.set_index(['DR_ID',"MR"], inplace=True)
visit_by_doc_df

DR_NAME            CHECK_IN  DURATION
DR_ID MR                                                      
1001  77650  Sheela Morley, M.D. 2023-05-17 13:50:00      1.25
      77650  Sheela Morley, M.D. 2023-05-31 14:00:00      0.75
1002  89765   Malcolm Head, D.O. 2023-06-04 10:00:00      0.50
1003  12446  Patricia Lord, M.D. 2023-05-31 10:15:00      0.10
      12446  Patricia Lord, M.D. 2023-06-04 10:15:00      0.10

In [19]:
visit_by_doc_df.join(patients_df[["LASTNM", "FIRSTNM", "PHONE"]], 
                        on="MR")

DR_NAME            CHECK_IN  DURATION     LASTNM  \
DR_ID MR                                                                    
1001  77650  Sheela Morley, M.D. 2023-05-17 13:50:00      1.25   Flemming   
      77650  Sheela Morley, M.D. 2023-05-31 14:00:00      0.75   Flemming   
1002  89765   Malcolm Head, D.O. 2023-06-04 10:00:00      0.50  Constanza   
1003  12446  Patricia Lord, M.D. 2023-05-31 10:15:00      0.10     Turing   
      12446  Patricia Lord, M.D. 2023-06-04 10:15:00      0.10     Turing   

            FIRSTNM         PHONE  
DR_ID MR                           
1001  77650   Bruce  503-311-9928  
      77650   Bruce  503-311-9928  
1002  89765   Susan  503-321-8640  
1003  12446   Jerry  503-311-7865  
      12446   Jerry  503-311-7865

Lets [create a SQLite database](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_sql.html?highlight=to_sql#pandas.DataFrame.to_sql) from the DataFrames we have so far.

In [20]:
import sqlite3 as sql

In [21]:
conn_db = sql.connect("practice") # a physicians' practive

In [22]:
patients_df.to_sql("patients", con=conn_db, if_exists='replace')
visits_df.to_sql("visits", con=conn_db, if_exists='replace')
physicians_df.to_sql("physicians", con=conn_db, if_exists='replace')

conn_db.close()

In [23]:
conn_db = sql.connect("practice") # a physicians' practive

curs = conn_db.cursor()
curs.execute("SELECT * FROM patients;")
result = list(curs.fetchall())
conn_db.close()

result[0:2]

[('13298',
  'Debbie',
  'Rose',
  '32 SE Beacon St.',
  'Portland',
  'OR',
  '97214',
  '503-311-9928'),
 ('12446',
  'Jerry',
  'Turing',
  '491 NW Shanny St.',
  'Portland',
  'OR',
  '97111',
  '503-311-7865')]

In [24]:
conn_db = sql.connect("practice") # a physicians' practive

curs = conn_db.cursor()
curs.execute("""SELECT * FROM visits""")
result = list(curs.fetchall())
conn_db.close()

result

[(0, '77650', '1001', '2023-05-17 13:50:00', 1.25),
 (1, '77650', '1001', '2023-05-31 14:00:00', 0.75),
 (2, '12446', '1003', '2023-05-31 10:15:00', 0.1),
 (3, '89765', '1002', '2023-06-04 10:00:00', 0.5),
 (4, '12446', '1003', '2023-06-04 10:15:00', 0.1)]

In [25]:
conn_db = sql.connect("practice") # a physicians' practive
curs = conn_db.cursor()

curs.execute("""SELECT visits.mr, lastnm, firstnm, dr_id, check_in, duration
                    FROM visits, patients
                    WHERE visits.mr == patients.mr""")
result = list(curs.fetchall())

conn_db.close()
result

[('77650', 'Flemming', 'Bruce', '1001', '2023-05-17 13:50:00', 1.25),
 ('77650', 'Flemming', 'Bruce', '1001', '2023-05-31 14:00:00', 0.75),
 ('12446', 'Turing', 'Jerry', '1003', '2023-05-31 10:15:00', 0.1),
 ('89765', 'Constanza', 'Susan', '1002', '2023-06-04 10:00:00', 0.5),
 ('12446', 'Turing', 'Jerry', '1003', '2023-06-04 10:15:00', 0.1)]

SELECT a1, a2, b1, b2
FROM A
INNER JOIN B on B.f = A.f;

In [26]:
conn_db = sql.connect("practice") # a physicians' practive
curs = conn_db.cursor()

curs.execute("""SELECT visits.mr, lastnm, firstnm, dr_id, check_in, duration
                    FROM visits
                    INNER JOIN patients on visits.mr = patients.mr""")
result = list(curs.fetchall())

conn_db.close()
result

[('77650', 'Flemming', 'Bruce', '1001', '2023-05-17 13:50:00', 1.25),
 ('77650', 'Flemming', 'Bruce', '1001', '2023-05-31 14:00:00', 0.75),
 ('12446', 'Turing', 'Jerry', '1003', '2023-05-31 10:15:00', 0.1),
 ('89765', 'Constanza', 'Susan', '1002', '2023-06-04 10:00:00', 0.5),
 ('12446', 'Turing', 'Jerry', '1003', '2023-06-04 10:15:00', 0.1)]

In [27]:
conn_db = sql.connect("practice") # a physicians' practive
curs = conn_db.cursor()

curs.execute("""SELECT patients.mr, lastnm, firstnm, dr_id, check_in, duration
                    FROM patients
                    LEFT JOIN visits on visits.mr = patients.mr""")
result = list(curs.fetchall())

conn_db.close()
result

[('13298', 'Rose', 'Debbie', None, None, None),
 ('12446', 'Turing', 'Jerry', '1003', '2023-05-31 10:15:00', 0.1),
 ('12446', 'Turing', 'Jerry', '1003', '2023-06-04 10:15:00', 0.1),
 ('77650', 'Flemming', 'Bruce', '1001', '2023-05-17 13:50:00', 1.25),
 ('77650', 'Flemming', 'Bruce', '1001', '2023-05-31 14:00:00', 0.75),
 ('89765', 'Constanza', 'Susan', '1002', '2023-06-04 10:00:00', 0.5),
 ('56768', 'Sosa', 'Raul', None, None, None)]